#### Saturday, December 14, 2024

This notebook was not part of the original authors repository, but was created from the sample code from Chapter 3.

I'm really kinda surprised the authors repository didn't include a notebook!

In [1]:
from openai import OpenAI
import os

# Set your OpenAI key as an environment variable
# https://platform.openai.com/api-keys
# client = OpenAI(
#   api_key=os.environ['OPENAI_API_KEY'],  # Default
# )

# Point to the local server ... last guy wins.
# lmstudio = "http://localhost:1234/v1"
lmstudio = "http://192.168.2.16:1234/v1"

client = OpenAI(base_url=lmstudio, api_key="lm-studio")

model = "qwen2.5-14b-instruct" # lmstudio-community/Qwen2.5-14B-Instruct-GGUF : wen2.5-14B-Instruct-Q8_0.gguf

# model = "hermes-3-llama-3.2-3b"


In [2]:
def get_response(prompt):
    response = client.chat.completions.create(
        # model="gpt-3.5-turbo",
        model = model ,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return response.choices[0].message.content

### Generating Lists

In [3]:
prompt = """Generate a list of Disney characters."""

In [4]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 18.5s
# 17.8s

Here is a diverse list of popular Disney characters from various movies and shows:

1. Mickey Mouse (Steamboat Willie)
2. Minnie Mouse (Steamboat Willie)
3. Donald Duck (The Wise Little Hen)
4. Daisy Duck (Don Donald)
5. Goofy (Mickey's Revue)
6. Pluto (Pluto's Judgement Day)
7. Scrooge McDuck (Christmas on Bear Mountain, Christmas at Willow Bay)
8. Jiminy Cricket (Pinocchio)
9. Dopey (Snow White and the Seven Dwarfs)
10. Snow White (Snow White and the Seven Dwarfs)
11. Cinderella (Cinderella)
12. Aurora / Sleeping Beauty (Sleeping Beauty)
13. Belle (Beauty and the Beast)
14. Jasmine (Aladdin)
15. Pocahontas (Pocahontas)
16. Mulan (Mulan)
17. Tiana (The Princess and the Frog)
18. Rapunzel (Tangled)
19. Merida (Brave)
20. Moana (Moana)

Animated Movie Characters:
1. Simba, Nala, Pumbaa, Timon, Rafiki (The Lion King)
2. Stitch (Lilo & Stitch)
3. Buzz Lightyear and Woody (Toy Story series)
4. Sully and Mike Wazowski (Monsters, Inc.)
5. Baloo, Mowgli (The Jungle Book)
6. Nemo and Marlin (F

The response is unpredictable because we were not clear about the type of list we wanted. Let's try again, this time being more specific with what we want.

In [5]:
prompt = """Generate a bullet point list of 5 male Disney characters. 
Only include the name of the character for each line. 
Never include the film for each Disney character.

Below is an example list:
* Aladdin
* Simba
* Beast
* Hercules
* Tarzan"""

In [ ]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 0.9s
# 0.7s

* Peter Pan
* Genie
* Woody
* Mowgli
* Stitch


It's interesting to observe just how much faster this prompt runs!

### Hierarchical List Generation

Hierarchical lists are useful for when your desired output is nested. A good example of this would be a detailed article structure.

In [7]:
prompt = """Generate a hierarchical and incredibly detailed article outline on "what are the benefits of data engineering".

See an example of the hierarchical structure below:

Article Title: What are the benefits of digital marketing?

* Introduction
    a. Explanation of digital marketing
    b. Importance of digital marketing in today’s business world

* Increased Brand Awareness
    a. Definition of brand awareness
    b. How digital marketing helps in increasing brand awareness"""

In [ ]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 24.2s
# 34.9s

### Article Title: What Are the Benefits of Data Engineering?

#### I. Introduction
   - A. Explanation of data engineering
       1. Definition and scope of data engineering
       2. Evolution of data engineering
   - B. Importance of data engineering in today’s business world
       1. Role in modern business operations
       2. Impact on decision-making processes

#### II. Enhanced Data Management Capabilities
   - A. Efficient Data Collection and Integration
       1. Automation of data ingestion from various sources
       2. Standardization and normalization of diverse datasets
   - B. Scalability and Flexibility
       1. Adapting to growing data volumes and changing business needs
       2. Elastic computing resources for optimal performance

#### III. Improved Data Quality and Reliability
   - A. Data Cleaning and Validation Techniques
       1. Identifying and removing inconsistencies or inaccuracies
       2. Implementing robust validation rules
   - B. Ensuring Consistenc

So you’ve successfully produced a hierarchical article outline, but how could you parse the string into structured data?

Example 3.1 Parsing a hierierarchical list 

In [9]:
import re

# openai_result = generate_article_outline(prompt)

openai_result = """
* Introduction
    a. Explanation of data engineering
    b. Importance of data engineering in today’s data-driven world
* Efficient Data Management
    a. Definition of data management
    b. How data engineering helps in efficient data management.
* Conclusion
    a. Importance of Data Engineering in the modern business world
    b. Future of Data Engineering and its impact on the data ecosystem
"""

# Regular expression patterns
heading_pattern = r"\* (.+)"
subheading_pattern = r"\s+[a-z]\. (.+)"

# Extract headings and subheadings
headings = re.findall(heading_pattern, openai_result)
subheadings = re.findall(subheading_pattern, openai_result)

# Print results
print("Headings:\n")
for heading in headings:
    print(f"* {heading}")

print("\nSubheadings:\n")
for subheading in subheadings:
    print(f"* {subheading}")

Headings:

* Introduction
* Efficient Data Management
* Conclusion

Subheadings:

* Explanation of data engineering
* Importance of data engineering in today’s data-driven world
* Definition of data management
* How data engineering helps in efficient data management.
* Importance of Data Engineering in the modern business world
* Future of Data Engineering and its impact on the data ecosystem


Example 3.2 Parsing a hierarchical list into a Python dictionary:

In [12]:
import re

openai_result = """
* Introduction
  a. Explanation of data engineering
  b. Importance of data engineering in today’s data-driven world
* Efficient Data Management
    a. Definition of data management
    b. How data engineering helps in efficient data management.
    c. Why data engineering is important for data management
* Conclusion
    a. Importance of Data Engineering in the modern business world
    b. Future of Data Engineering and its impact on the data ecosystem
"""

section_regex = re.compile(r"\* (.+)")
subsection_regex = re.compile(r"\s*([a-z]\..+)")

result_dict = {}
current_section = None

for line in openai_result.split("\n"):
    section_match = section_regex.match(line)
    subsection_match = subsection_regex.match(line)

    if section_match:
        current_section = section_match.group(1)
        result_dict[current_section] = []
    elif subsection_match and current_section is not None:
        result_dict[current_section].append(subsection_match.group(1))

#print(result_dict)
result_dict

{'Introduction': ['a. Explanation of data engineering',
  'b. Importance of data engineering in today’s data-driven world'],
 'Efficient Data Management': ['a. Definition of data management',
  'b. How data engineering helps in efficient data management.',
  'c. Why data engineering is important for data management'],
 'Conclusion': ['a. Importance of Data Engineering in the modern business world',
  'b. Future of Data Engineering and its impact on the data ecosystem']}

### When to Avoid using Regular Expressions

#### Generating JSON

In [13]:
prompt = """Compose a very detailed article outline on "The benefits of learning code" with a JSON payload structure that highlights key points.

Only return valid JSON.

Here is an example of the JSON structure:
```{
    "Introduction": [
        "a. Explanation of data engineering",
        "b. Importance of data engineering in today’s data-driven world"],
    "Efficient Data Management": [
        "a. Definition of data management",
        "b. How data engineering helps in efficient data management."],
    "Conclusion": [
        "a. Importance of Data Engineering in the modern business world",
        "b. Future of Data Engineering and its impact on the data ecosystem"]
} """

In [14]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 17.1s

```json
{
    "Introduction": [
        "a. Explanation of what coding is and why it's important",
        "b. Overview of the increasing demand for coders in various industries"
    ],
    "Enhanced Problem-Solving Skills": [
        "a. Development of logical thinking through programming challenges",
        "b. Application of problem-solving skills to real-world scenarios beyond coding",
        "c. Improved analytical capabilities and decision-making processes"
    ],
    "Career Opportunities and Advancement": [
        "a. Broad range of career options in software development, data science, web design, etc.",
        "b. High demand for skilled coders leading to better job prospects and higher salaries",
        "c. Continuous learning and adaptation necessary in the rapidly evolving tech industry"
    ],
    "Personal Creativity and Innovation": [
        "a. Freedom to create unique software applications or improve existing ones",
        "b. Collaboration with other developers

Now let's examine how we can parse JSON output with Python:

In [16]:
import json

# openai_json_result = generate_article_outline(prompt)

openai_json_result = """
{
    "Introduction": [
        "a. Overview of coding and programming languages",
        "b. Importance of coding in today's technology-driven world"],
    "Conclusion": [
        "a. Recap of the benefits of learning code",
        "b. The ongoing importance of coding skills in the modern world"]
}
"""

parsed_json_payload = json.loads(openai_json_result)
# print(parsed_json_payload)
parsed_json_payload

{'Introduction': ['a. Overview of coding and programming languages',
  "b. Importance of coding in today's technology-driven world"],
 'Conclusion': ['a. Recap of the benefits of learning code',
  'b. The ongoing importance of coding skills in the modern world']}

#### YAML

In [17]:
prompt = """Return valid `.yml` for 5 apple slices, and 2 dozen eggs.

- Below you'll find the current yaml schema.
- Please only use the schema below and if you are unsure, then return `Unsure`.

```
- item: Apple Slices
  quantity: 5
  unit: pieces
- item: Milk
  quantity: 1
  unit: gallon
- item: Bread
  quantity: 2
  unit: loaves
- item: Eggs
  quantity: 1
  unit: dozen
```"""

In [18]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 2.4s

Based on the provided schema, here is the valid `.yml` for 5 apple slices and 2 dozen eggs:

```
- item: Apple Slices
  quantity: 5
  unit: pieces
- item: Eggs
  quantity: 2
  unit: dozen
```


Filtering YAML payloads

In [19]:
import yaml


class InvalidResponse(Exception):
    pass


class InvalidItemType(Exception):
    pass


class InvalidItemKeys(Exception):
    pass


class InvalidItemName(Exception):
    pass


class InvalidItemQuantity(Exception):
    pass


class InvalidItemUnit(Exception):
    pass


# Provided schema
schema = """
- item: Apple Slices
  quantity: 5
  unit: pieces
- item: Milk
  quantity: 1
  unit: gallon
- item: Bread
  quantity: 2
  unit: loaves
- item: Eggs
  quantity: 1
  unit: dozen
"""


def validate_response(response, schema):
    # Parse the schema
    schema_parsed = yaml.safe_load(schema)

    # Check if the response is a list
    if not isinstance(response, list):
        raise InvalidResponse("Response is not a list")

    # Check if each item in the list is a dictionary
    for item in response:
        if not isinstance(item, dict):
            raise InvalidItemType("Response item is not a dictionary")

        # Check if each dictionary has the keys "item", "quantity", and "unit"
        if not all(key in item for key in ("item", "quantity", "unit")):
            raise InvalidItemKeys("Response item does not have the correct keys")

        # Check if the values associated with each key are the correct type
        if not isinstance(item["item"], str):
            raise InvalidItemName("Response item name is not a string")
        if not isinstance(item["quantity"], int):
            raise InvalidItemQuantity("Response item quantity is not an integer")
        if not isinstance(item["unit"], str):
            raise InvalidItemUnit("Response item unit is not a string")

        # Check if the values associated with each key are the correct value
        if item["item"] not in [x["item"] for x in schema_parsed]:
            raise InvalidItemName("Response item name is not in schema")
        if item["quantity"] > 10:
            raise InvalidItemQuantity("Response item quantity is greater than 10")
        if item["unit"] not in ["pieces", "dozen"]:
            raise InvalidItemUnit("Response item unit is not pieces or dozen")


# Fake responses
fake_response_1 = """
- item: Apple Slices
  quantity: 5
  unit: pieces
- item: Eggs
  quantity: 2
  unit: dozen
"""

fake_response_2 = """
# Updated yaml list
- item: Apple Slices
  quantity: 5
  unit: pieces
"""

fake_response_3 = """Unmatched"""

# Parse the fake responses
response_1_parsed = yaml.safe_load(fake_response_1)
response_2_parsed = yaml.safe_load(fake_response_2)
response_3_parsed = yaml.safe_load(fake_response_3)

# Validate the responses against the schema
try:
    validate_response(response_1_parsed, schema)
    print("Response 1 is valid")
except Exception as e:
    print("Response 1 is invalid:", str(e))

try:
    validate_response(response_2_parsed, schema)
    print("Response 2 is valid")
except Exception as e:
    print("Response 2 is invalid:", str(e))

try:
    validate_response(response_3_parsed, schema)
    print("Response 3 is valid")
except Exception as e:
    print("Response 3 is invalid:", str(e))

Response 1 is valid
Response 2 is valid
Response 3 is invalid: Response is not a list


#### Diverse Format Generation with ChatGPT

I am going to restart things from here just so that I do not have to run anything above ...

In [1]:
from openai import OpenAI
import os

# Point to the local server ... last guy wins.
lmstudio = "http://localhost:1234/v1"

client = OpenAI(base_url=lmstudio, api_key="lm-studio")

model = "qwen2.5-14b-instruct" # lmstudio-community/Qwen2.5-14B-Instruct-GGUF : Qwen2.5-14B-Instruct-Q4_K_M.gguf



Generally, the default temperature setting in OpenAI’s fine-tuned models is around 0.7. Temperature controls the randomness of the generated text. Lower temperatures result in more deterministic output, while higher temperatures lead to more diversity and randomness in the generated text.

In [2]:
def get_response(prompt):
    response = client.chat.completions.create(
        # model="gpt-3.5-turbo",
        model = model ,
        temperature=0.0, # Setting the temperature to 0 ensures that the model will always choose the most likely next word, leading to more deterministic and less random output.
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return response.choices[0].message.content

In [3]:
prompt = """Create a flowchart diagram in mermaid syntax showing a simple food ordering
process that starts with "Choose food" and ends with "Pay for meal."
Include steps for "Add food to cart" and "Confirm cart."`"""

In [ ]:
response = get_response(prompt)
print(response)

# 49.8 s

Certainly! Below is the Mermaid syntax for a flowchart depicting a simple food ordering process:

```mermaid
flowchart TD

    A[Choose food] --> B{Do you want to add more?}
    B -- Yes --> C[Add food to cart]
    C --> D{Confirm cart?}
    D -- No --> B
    D -- Yes --> E[Pay for meal]

    style A fill:#f96,stroke:#333,stroke-width:4px
    style E fill:#6c6,stroke:#333,stroke-width:4px
```

### Explanation:
- **A**: The process starts with "Choose food."
- **B**: After choosing the food, there's a decision point asking if you want to add more items.
  - If yes, it goes back to adding more food to the cart (`C`).
  - If no, it proceeds to confirm the cart (`D`).
- **C**: The selected food is added to the cart.
- **D**: There's a confirmation step asking if you want to proceed with the current items in the cart.
  - If not confirmed, it loops back to adding more food or confirming again.
  - If confirmed, it proceeds to paying for the meal (`E`).
- **E**: The process ends by paying fo

In [ ]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 1m 7.5s

Certainly! Below is the Mermaid syntax for a flowchart depicting a simple food ordering process:

```mermaid
flowchart TD

    A[Choose food] --> B{Do you want to add more?}
    B -->|Yes| C[Add food to cart]
    C --> D[Confirm cart]
    D --> E{Proceed to checkout?}
    E -->|Yes| F[Pay for meal]
    E -->|No| G[Remove items from cart]
    G --> H[Confirm cart]
    H --> I{Proceed to checkout?}
    I -->|Yes| F
    I -->|No| B

    B -->|No| D
```

### Explanation:
1. **A (Choose food)**: The process starts with the user choosing their desired food items.
2. **B (Do you want to add more?)**: After selecting an item, the system asks if the user wants to add more items to their order.
3. **C (Add food to cart)**: If the user chooses "Yes," they can add additional items to their cart.
4. **D (Confirm cart)**: The user confirms the items in their cart before proceeding further.
5. **E (Proceed to checkout?)**: After confirming the cart, the system asks if the user is ready to proceed to 

#### Mock CSV Data

In [6]:
prompt = """Generate a sample CSV data for five students with the following fields:
name, age, grade."""

In [7]:
 # Get a response from the model
response = get_response(prompt)
print(response)

Certainly! Below is an example of how you can structure a CSV file containing information about five students:

```csv
name,age,grade
Alice Smith,15,9
Bob Johnson,16,10
Charlie Brown,14,8
David Lee,17,12
Eve Wilson,15,9
```

Here's a breakdown of the data:
- Each line represents one student.
- The first line contains the column headers: `name`, `age`, and `grade`.
- Subsequent lines contain the actual data for each student.

You can save this content in a file with a `.csv` extension to use it as a CSV file.


#### Explain it like I'm five

In [8]:
prompt = """Explain the following text as if I was 5 years old. Explain it like I'm 5.
Text: This is a review regarding different types of cancer treatments.
We aimed at analyzing the tumor microenvironment and the recent trends
for the therapeutic applications and effectiveness for several kinds
of cancers. Traditionally the cancer treatment was based on the
neoplastic cells."""

In [9]:
 # Get a response from the model
response = get_response(prompt)
print(response)

Okay, little one! Let's talk about how doctors help people who are sick with something called "cancer."

Imagine you have a big coloring book where some pages have spots that shouldn't be there - those spots are like cancer in our bodies. Doctors used to just look at the spots and try to make them go away.

But now, they're doing something new! They're looking at the whole page of your coloring book, not just the spots. They want to know about all the colors around the spots too, because sometimes those other colors can help or hurt the spots.

The doctors are trying to find better ways to make the bad spots go away by understanding everything on the page, not just the spots themselves. This helps them figure out which crayons (or medicines) work best for different kinds of coloring books (different types of cancer).

So instead of just looking at one thing, they're looking at lots of things together to help people feel better!


#### Ask for Context

In [10]:
prompt = """Should I use MongoDB or PostgreSQL for my project?"""

In [11]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 1m 37.1s

Choosing between MongoDB and PostgreSQL depends on the specific requirements of your project, such as data structure, performance needs, scalability, and development team expertise. Here’s an overview to help you decide:

### MongoDB (NoSQL Document Database)
**Pros:**
1. **Flexible Schema:** Documents can have varying structures, making it easy to handle evolving data models.
2. **Scalability:** Horizontal scaling is straightforward with sharding and replication.
3. **Performance:** Good for high write throughput and complex queries involving large datasets.
4. **Aggregation Framework:** Powerful tool for querying and analyzing data.

**Cons:**
1. **ACID Compliance:** Not fully ACID-compliant (though MongoDB 4.0+ has improved this).
2. **Learning Curve:** If your team is more familiar with SQL, there might be a learning curve.
3. **Data Integrity:** Less robust in enforcing referential integrity compared to relational databases.

### PostgreSQL (Relational Database)
**Pros:**
1. **ACI

In [13]:
prompt = """Should I use MongoDB or PostgreSQL for my project?
Provide a list of requirements that would help you to make an informed
and high-quality decision about which database solution I should choose?"""

In [14]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 2m 0.6s

To determine whether MongoDB (a NoSQL document-oriented database) or PostgreSQL (an SQL relational database) is the right choice for your project, it's important to consider several key factors. Here’s a list of requirements and considerations that will help you make an informed decision:

### 1. **Data Model Complexity**
   - **Document-Oriented vs Relational Data Models:**
     - Does your data have complex hierarchical structures or nested documents?
     - Do you need flexible schema design with dynamic fields?

### 2. **Scalability Requirements**
   - **Horizontal Scaling (Sharding):**
     - Will the database need to scale horizontally by distributing data across multiple servers?
     - Are there specific requirements for handling large volumes of unstructured or semi-structured data?

### 3. **Performance and Speed**
   - **Query Performance:**
     - What are your performance expectations in terms of read/write speeds, latency, and throughput?
     - Do you need real-time anal

In [15]:
prompt = """Should I use MongoDB or PostgreSQL for my project?
Context:
- Data Structure: Highly unstructured data with varying document formats
and no fixed schema. Scalability: Project requires the ability to handle
massive amounts of data and distribute it across multiple servers.
- ACID Compliance: Strong data consistency and integrity are crucial for
the project's financial transactions.
- Querying Capabilities: Complex geospatial queries and aggregations on
location-based data.
Community and Ecosystem: Active community with numerous third-party
libraries and tools specifically tailored for the project's industry.
- Data Integrity and Constraints: Enforcing strict foreign key
relationships and complex data validation rules.
- Transactions and Concurrency: Requires handling high levels of
concurrent read and write operations while maintaining data consistency.
- Maturity and Stability: Prioritizing a well-established and extensively
tested database for long-term stability.
- Development and Operational Ease: Agile development approach with a need
for rapid prototyping and iterative schema design.
- Cost: Budget constraints necessitate using an open-source solution with
no additional licensing or support costs.
If you need more context, please specify what would help you to make
a better decision."""

In [16]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 1m 51.9s

Given the detailed requirements for your project, here's a breakdown of how MongoDB and PostgreSQL compare based on each criterion:

### Data Structure:
- **MongoDB**: Highly unstructured data with varying document formats fits well into MongoDB’s schema-less design.
- **PostgreSQL**: Can handle complex schemas but might require more upfront planning.

### Scalability:
- **MongoDB**: Known for its horizontal scalability, allowing easy distribution of data across multiple servers.
- **PostgreSQL**: Also scalable but typically requires more configuration and management to distribute data effectively.

### ACID Compliance:
- **MongoDB**: MongoDB 4.0+ supports multi-document transactions (ACID-compliant), but it's relatively new compared to PostgreSQL.
- **PostgreSQL**: Strongly adheres to ACID principles, making it a reliable choice for financial transactions.

### Querying Capabilities:
- **MongoDB**: Excellent support for geospatial queries and aggregations through its native indexing a

#### Text Style Unbundling

In [17]:
prompt = """Analyze the following text and identify core features that will allow
further documents to imitate the same style, consistency, and formatting
as the document below.
Provide all of the features as a style writing guide that will be used
for future documents.
Analyze the following text:
Artificial intelligence is rapidly transforming the world around us,
opening up new possibilities and revolutionizing industries. From
health care to entertainment, AI is making our lives better and more
convenient in countless ways. However, with great power comes great
responsibility. We must ensure that AI technologies are developed and
deployed ethically and responsibly to protect our privacy and uphold our
values."""

In [18]:
 # Get a response from the model
response = get_response(prompt)
print(response)

# 1m 13.9s

To imitate the style, consistency, and formatting of the given text, here is a detailed guide for future documents:

### Style Writing Guide

#### 1. **Tone**
   - The tone should be informative yet engaging.
   - Use a balanced perspective that acknowledges both the benefits and potential risks of AI.

#### 2. **Structure**
   - Start with an introductory sentence or two to set the context.
   - Follow with supporting sentences that elaborate on specific examples or areas where AI is making an impact.
   - Conclude with a statement about ethical considerations and responsibilities.

#### 3. **Sentence Structure**
   - Use a mix of short and long sentences for readability and emphasis.
   - Begin sentences with strong, declarative statements to capture attention.
   - Include transitional phrases (e.g., "From," "However") to maintain flow and coherence.

#### 4. **Vocabulary**
   - Employ clear and concise language.
   - Use technical terms related to AI when necessary but ensure they 

#### Extracting Specific Textual Features with LLMs

In [19]:
prompt  = """Analyze the following text to identify and list the entities mentioned:
Artificial intelligence is rapidly transforming the world around us,
opening up new possibilities and revolutionizing industries. From
health care to entertainment, AI is making our lives better and more
convenient in countless ways. However, with great power comes great
responsibility. We must ensure that AI technologies are developed
and deployed ethically and responsibly to protect our privacy and
uphold our values."""

In [20]:
 # Get a response from the model
response = get_response(prompt)
print(response)

The text mentions the following entities:

1. **Artificial intelligence (AI)** - The primary subject of discussion in the text.
2. **Health care** - One of the industries mentioned as being impacted by AI.
3. **Entertainment** - Another industry mentioned as being influenced by AI.

There are no specific named individuals, organizations, or locations mentioned in this passage.


#### Summarization

In [21]:
prompt = """Summarize the following text:
Whereas the party of the first part, hereinafter referred to as the
"Purchaser," and the party of the second part, hereinafter referred to
as the "Vendor," being fully cognizant of their respective legal
obligations and responsibilities pursuant to the applicable statutory
provisions and common law doctrines, have duly entered into a legally
binding agreement, hereinafter referred to as the "Agreement," for the
purpose of effectuating a transaction involving the transfer of goods,
hereinafter referred to as the "Goods," between the Purchaser and the
Vendor, subject to the terms and conditions set forth herein, the Parties
do hereby solemnly acknowledge and affirm the following: """

In [22]:
 # Get a response from the model
response = get_response(prompt)
print(response)

The text is an introductory legal statement in which:

- The "Purchaser" and "Vendor" agree to a binding contract.
- They are aware of their legal obligations under relevant laws.
- The agreement is for transferring goods from Vendor to Purchaser.
- The terms and conditions of the transaction are outlined.

In essence, it establishes that both parties acknowledge and accept the legal agreement for the sale of goods.


#### Summarizing Given Context Window Limitations